In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pytesseract
import cv2
import numpy as np

In [ ]:
#image = "../input_data/La Razón 1985-06-08 Un informe de....tif"
image = "../input_data/La Razón 1985-05-14 Declaró el padre....tif"
my_cfg = r"-l spa --psm 3 --oem 3"
custom_config = r'-c tessedit_char_whitelist="AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZaábcdeéfghiíjklmnñoópqrstuúvwxyz0123456789 -_/.,:;()"'
my_cfg_2 = r'-l spa --psm 1 --oem 3 -c tessedit_char_whitelist="AÁBCDEÉFGHIÍJKLMNÑOÓPQRSTUÚVWXYZaábcdeéfghiíjklmnñoópqrstuúvwxyz0123456789 -_/.,:;()"'
image_cv = cv2.imread(image)
plt.imshow(image_cv)

In [ ]:
df_data = pytesseract.image_to_data(image_cv, config=custom_config, output_type=pytesseract.Output.DATAFRAME)
df_data.info()

In [ ]:
def blank_filter(df_data):
    """Devuelve df sin elementos que fueron detectados como espacios. Parámetro: df original."""   
    mask_not_blank = (df_data['text'].str.strip() != '')
    #print('Eliminados ', df_data[~mask_not_blank].shape[0])
    #print(df_data[~mask_not_blank])
    df_data = df_data[mask_not_blank].dropna(subset=['text'])
    return df_data

In [ ]:
df_data = blank_filter(df_data)

In [ ]:
df_data.info()

In [ ]:
for token in df_data.iterrows():
    x,y,w,h = token[1].left, token[1].top, token[1].width, token[1].height
    cv2.rectangle(image_cv, (x, y), (x + w, y + h), (0, 255, 0), 3)

#cv2.imwrite(f"{foto}_out_bloques_img", imagen_cv)
plt.figure(figsize = (50,50))
plt.axis('off')
plt.imshow(image_cv)

In [ ]:
def get_tokens_groups(df_data):
    df_data['x2'] = df_data['left'] + df_data['width']
    df_data['y2'] = df_data['top'] + df_data['height']

    df_data['id_line_group'] = df_data.apply(lambda row: str(row['block_num']) + str(row['par_num']) + str(row['line_num']), axis=1)

    ids_line_groups = df_data['id_line_group'].value_counts().keys().to_list()
    index_line_groups = [x for x in range(0, len(ids_line_groups))]

    df_data['id_line_group'].replace(ids_line_groups, index_line_groups, inplace=True)
    return df_data

In [ ]:
groups_boundaries = {}
for g in index_line_groups:
    group_boundaries = {'x1': 0, 'y1': 0, 'x2': 0, 'y2': 0}
    df_g = df_data[df_data.id_line_group == g]
    group_boundaries['x1'] = df_g['left'].min()
    group_boundaries['y1'] = df_g['top'].min()
    group_boundaries['x2'] = df_g['x2'].max()# + df_g.loc[df_g['left'].idxmax()]['width']
    group_boundaries['y2'] = df_g['y2'].max()# + df_g.loc[df_g['top'].idxmax()]['height']
    groups_boundaries[g] = group_boundaries
    

In [ ]:
groups_boundaries

In [ ]:
for g in groups_boundaries:
    x1,y1,x2,y2 = groups_boundaries[g]['x1'], groups_boundaries[g]['y1'], groups_boundaries[g]['x2'], groups_boundaries[g]['y2']
    cv2.rectangle(image_cv, (x1, y1), (x2, y2), (255, 0, 0), 2)

#cv2.imwrite(f"{foto}_out_bloques_img", imagen_cv)
plt.figure(figsize = (50,50))
plt.axis('off')
plt.imshow(image_cv)

In [ ]:
df_data.groupby(by=['block_num', 'line_num']).count()